## Libraries

In [146]:
# !pip3 install pymongo

In [417]:
import pymongo
from pymongo import MongoClient
from multiprocessing import Pool
import pandas as pd
import numpy as np
import pprint

# B1

Based on the two data sets provided i.e. Fire data-Part 1 and Weather data-Part 1, design a suitable data model to support efficient querying of the two data sets in
MongoDB. Justify your data model design

### Model:

The two datasets represent two differnt things about a date. The climate data provides weather details of the particulate day and provides the station number as to identify which station provided the details for the weather. The fire data provides the details of a fire if a fire has occured on a particular day. 

We can join the two dataframes to essentially provide data for each day with details of its weather and within the same data, a list of fires if they have occured in the dataframe. Since the fires do not occur every day and only occur on some days we can make use of the non-schema structure of the mongoDB. Which this we can essentially create a model in which we can just add the fire data in the date if the fire does occur. In case the fire does not occur, we can model it in two ways:

* Add a variable in the collection object and add a value NULL to it in-order to signify the absense of that field. 
* Leave the variable absent in the collection object to signify it not occurring. 

In this model, the absense of the variable is kept as a way to show the non-occurance of that variable. This lets us take better advantage of the document like structure of the mongoDB. 

Key notes in our data:
* The dates in climate data cover all the dates in fire data. i.e there are no dates in fire data that do not occur in climate.
* Fire data does not include all dates, since it didnt occur in all dates.
* There are cases where there are multiple fires on the same date.
* Temperature in fire data is provided in Kelvin and Celcius however weather related temperatures in Climate are only mentioned in Celcius. 
* Locations of the fire are provided with two variables Lattitude, and Longitude
* Both datasets include the date column which can used a key to join the datasets.
* When the datasets would be joined on date there will be fire data missing for a few dates, hence, when making the model, we will have to make sure we can remove Fire occurances from the objects. 
* Since the weather data is for almost all dates the focus of the model would be on the weather information and would include the fire information if the fire does occur. 

Keeping the above mentioned points, each object in the collection would represent a date in our data and each different object would include a distinct date; to ensure this an index will be created on date with distint true. For that date the data for the weather would be provided with key value pairs except for Air Temperature which would have a value of an object to include temperature in celcius and kelvin. 

The fire data would be in the key fire and would be represented in array form, since there can be multiple fires in one day. Although the array form is not preferred compared to creating another object, however, there is no proper key that can be used for the instances of the fire and hence the array type would have to be created. The array would include an object to represent each instance of a fire on that day. Since the location is given in latitude and longitude form it would be represented as an object with keys latitude and longitude. Same goes for Surface Temperature since it includes values in Kelvin and Celcius. 

An example of the data model for an example date is provided below:


```python
{'AirTemperature': {'Celcius': 19, 'Kelvin': 66.2},
 'Date': '2017-03-07',
 'Fire': [{'Confidence': 88.0,
           'Location': {'Latitude': -37.7752, 'Longitude': 141.9086},
           'Power': 30.0,
           'SurfaceTemperature': {'Celcius': 64.0, 'Kelvin': 337.8},
           'Time': '04:16:10'}],
 'MAX': '80.4*',
 'MIN': '52.9*',
 'MaxWindSpeed': 20.0,
 'Precipitation': '0.00I',
 'RelativeHumidity': 51.5,
 'Station': 948701,
 'WindSpeedKnots': 10.2}
```

# B2

 Create a new database in MongoDB. The new database will be based on the document model you have designed in Task B1.

In [148]:
#connecting pymongo
client = MongoClient ()

In [149]:
#specifying the database
db = client.ass2_b

In [150]:
#specifying the collection
coll = db.model

# B3
Write a python program that will read the data from Fire data-Part1 and Climate data Part1 and load them to the new database created in Task B2.

### Joing DataFrames

In [151]:
#data import
fd = pd.read_csv('FireData-Part1.csv')
cd = pd.read_csv('ClimateData-Part1.csv')

In [152]:
# fixing white space issues with climate data
cd.columns = cd.columns.str.strip()

In [153]:
# joining the two dataframes
df = pd.merge(fd, cd, on='Date', how='outer')

### Cleaning data whitespace from data values

ref: https://stackoverflow.com/questions/40950310/strip-trim-all-strings-of-a-dataframe

In [154]:
df_obj = df.select_dtypes(['object'])
df[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())

### Creating Data

ref:https://stackoverflow.com/questions/9452775/converting-numpy-dtypes-to-native-python-types

In [155]:
#instantiate variable
data = []
db = {}
rec={}
prev_date = None

dates = set()

#sorting dataframe
df.sort_values(by='Date', inplace=True)
df = df.reset_index(drop=True)

#create data
for i in range(2889):
    
    #identify current date
    curr_date = df.loc[i, 'Date']
    
    dates.add(curr_date)
    
    #check if data has repeated (dates are ordered hence we can check only the prev one)
    if curr_date != prev_date:
        if db:
            #putting record in data
            data.append(db)
            db={}
        
        #fill in weather data
        db['Date'] = df.loc[i, 'Date']
        db['Station'] = df.loc[i, 'Station'].item()
        db['AirTemperature'] = {'Celcius':df.loc[i, 'Air Temperature(Celcius)'].item(), 'Kelvin':round(df.loc[i, 'Air Temperature(Celcius)'].item()*(9/5)+32,2)}
        db['RelativeHumidity'] = df.loc[i, 'Relative Humidity'].item()
        db['WindSpeedKnots'] = df.loc[i, 'WindSpeed  (knots)'].item()
        db['MaxWindSpeed'] = df.loc[i, 'Max Wind Speed'].item()
        db['MAX'] = df.loc[i, 'MAX']
        db['MIN'] = df.loc[i, 'MIN']
        db['Precipitation'] = df.loc[i, 'Precipitation']
    
    #fill in fire data if it exists
    #if not null
    if ~df.loc[i, ['Latitude']].isnull()[0]:
        
        #create list and rec dict
        db.setdefault('Fire', [])
        rec = {}
        
        #adding fire values
        rec['Location'] = {'Latitude':df.loc[i, 'Latitude'].item(), 'Longitude':df.loc[i, 'Longitude'].item()}
#         rec['Time'] = df.loc[i, 'Datetime'][-8:]
        rec['Power'] =  df.loc[i, 'Power'].item()
        rec['Confidence'] =  df.loc[i, 'Confidence'].item()
        rec['Confidence'] =  df.loc[i, 'Confidence'].item()
        rec['Datetime'] =  df.loc[i, 'Datetime']
        rec['SurfaceTemperature'] =  {'Celcius':df.loc[i, 'Surface Temperature (Celcius)'].item() , 'Kelvin':df.loc[i, 'Surface Temperature (kelvin)'].item()}
        
        #adding fire record to fire list in db
        db['Fire'].append(rec)
        rec={}
        
    #add last record to dataframe
    if i==2888:
        #putting last record in data
        data.append(db)
        db={}
    
    #store value in prev date
    prev_date = curr_date    
    


### Checking the data format created

In [156]:
pp = pprint.PrettyPrinter()
for rec in data:
    pp.pprint(rec)

{'AirTemperature': {'Celcius': 19, 'Kelvin': 66.2},
 'Date': '2016-12-31',
 'MAX': '72.0*',
 'MIN': '61.9*',
 'MaxWindSpeed': 11.1,
 'Precipitation': '0.00I',
 'RelativeHumidity': 56.8,
 'Station': 948700,
 'WindSpeedKnots': 7.9}
{'AirTemperature': {'Celcius': 15, 'Kelvin': 59.0},
 'Date': '2017-01-02',
 'MAX': '72.0*',
 'MIN': '58.3',
 'MaxWindSpeed': 13.0,
 'Precipitation': '0.02G',
 'RelativeHumidity': 50.7,
 'Station': 948700,
 'WindSpeedKnots': 9.2}
{'AirTemperature': {'Celcius': 16, 'Kelvin': 60.8},
 'Date': '2017-01-03',
 'MAX': '68.4*',
 'MIN': '54.3*',
 'MaxWindSpeed': 15.0,
 'Precipitation': '0.00G',
 'RelativeHumidity': 53.6,
 'Station': 948700,
 'WindSpeedKnots': 8.1}
{'AirTemperature': {'Celcius': 24, 'Kelvin': 75.2},
 'Date': '2017-01-04',
 'MAX': '87.3*',
 'MIN': '54',
 'MaxWindSpeed': 14.0,
 'Precipitation': '0.00I',
 'RelativeHumidity': 61.6,
 'Station': 948700,
 'WindSpeedKnots': 7.7}
{'AirTemperature': {'Celcius': 24, 'Kelvin': 75.2},
 'Date': '2017-01-05',
 'MAX': '

           'Power': 28.9,
           'SurfaceTemperature': {'Celcius': 67.0, 'Kelvin': 340.5}},
          {'Confidence': 81.0,
           'Datetime': '2017-03-14T04:14:40',
           'Location': {'Latitude': -34.3539, 'Longitude': 141.5629},
           'Power': 10.3,
           'SurfaceTemperature': {'Celcius': 55.0, 'Kelvin': 328.4}}],
 'MAX': '88.7*',
 'MIN': '59.9',
 'MaxWindSpeed': 13.0,
 'Precipitation': '0.00I',
 'RelativeHumidity': 58.3,
 'Station': 948701,
 'WindSpeedKnots': 7.1}
{'AirTemperature': {'Celcius': 28, 'Kelvin': 82.4},
 'Date': '2017-03-15',
 'Fire': [{'Confidence': 74.0,
           'Datetime': '2017-03-15T04:59:10',
           'Location': {'Latitude': -37.4058, 'Longitude': 147.0117},
           'Power': 18.5,
           'SurfaceTemperature': {'Celcius': 48.0, 'Kelvin': 321.7}},
          {'Confidence': 76.0,
           'Datetime': '2017-03-15T05:02:00',
           'Location': {'Latitude': -37.4108, 'Longitude': 147.0196},
           'Power': 20.9,
           'Sur

          {'Confidence': 67.0,
           'Datetime': '2017-03-26T04:39:10',
           'Location': {'Latitude': -37.1956, 'Longitude': 141.0252},
           'Power': 12.8,
           'SurfaceTemperature': {'Celcius': 44.0, 'Kelvin': 317.2}},
          {'Confidence': 74.0,
           'Datetime': '2017-03-26T04:39:10',
           'Location': {'Latitude': -38.0542, 'Longitude': 142.8655},
           'Power': 12.6,
           'SurfaceTemperature': {'Celcius': 48.0, 'Kelvin': 321.2}}],
 'MAX': '84.4*',
 'MIN': '64.2*',
 'MaxWindSpeed': 26.0,
 'Precipitation': '0.00I',
 'RelativeHumidity': 60.6,
 'Station': 948701,
 'WindSpeedKnots': 10.1}
{'AirTemperature': {'Celcius': 17, 'Kelvin': 62.6},
 'Date': '2017-03-27',
 'MAX': '91.8*',
 'MIN': '53.6*',
 'MaxWindSpeed': 25.1,
 'Precipitation': '0.16G',
 'RelativeHumidity': 53.4,
 'Station': 948701,
 'WindSpeedKnots': 9.4}
{'AirTemperature': {'Celcius': 18, 'Kelvin': 64.4},
 'Date': '2017-03-28',
 'Fire': [{'Confidence': 97.0,
           'Datetime'

           'SurfaceTemperature': {'Celcius': 45.0, 'Kelvin': 318.9}},
          {'Confidence': 71.0,
           'Datetime': '2017-04-01T04:04:50',
           'Location': {'Latitude': -36.1017, 'Longitude': 147.6381},
           'Power': 21.6,
           'SurfaceTemperature': {'Celcius': 46.0, 'Kelvin': 319.2}},
          {'Confidence': 77.0,
           'Datetime': '2017-04-01T04:02:50',
           'Location': {'Latitude': -36.1256, 'Longitude': 145.2444},
           'Power': 24.0,
           'SurfaceTemperature': {'Celcius': 53.0, 'Kelvin': 326.4}},
          {'Confidence': 58.0,
           'Datetime': '2017-04-01T04:02:40',
           'Location': {'Latitude': -36.2044, 'Longitude': 145.7706},
           'Power': 19.1,
           'SurfaceTemperature': {'Celcius': 51.0, 'Kelvin': 324.7}},
          {'Confidence': 73.0,
           'Datetime': '2017-04-01T04:11:30',
           'Location': {'Latitude': -36.1948, 'Longitude': 145.7688},
           'Power': 18.7,
           'SurfaceTemperatu

           'SurfaceTemperature': {'Celcius': 76.0, 'Kelvin': 349.4}},
          {'Confidence': 72.0,
           'Datetime': '2017-04-04T04:36:00',
           'Location': {'Latitude': -36.0694, 'Longitude': 141.5717},
           'Power': 19.4,
           'SurfaceTemperature': {'Celcius': 54.0, 'Kelvin': 327.3}},
          {'Confidence': 70.0,
           'Datetime': '2017-04-04T04:36:30',
           'Location': {'Latitude': -37.4465, 'Longitude': 143.5163},
           'Power': 27.7,
           'SurfaceTemperature': {'Celcius': 50.0, 'Kelvin': 324.1}},
          {'Confidence': 75.0,
           'Datetime': '2017-04-04T04:36:10',
           'Location': {'Latitude': -37.3632, 'Longitude': 145.8356},
           'Power': 32.9,
           'SurfaceTemperature': {'Celcius': 53.0, 'Kelvin': 326.4}},
          {'Confidence': 88.0,
           'Datetime': '2017-04-04T04:36:10',
           'Location': {'Latitude': -36.6078, 'Longitude': 141.9742},
           'Power': 36.9,
           'SurfaceTemperatu

...

 'Date': '2017-09-21',
 'Fire': [{'Confidence': 76.0,
           'Datetime': '2017-09-21T05:11:46',
           'Location': {'Latitude': -37.228, 'Longitude': 142.14600000000002},
           'Power': 60.2,
           'SurfaceTemperature': {'Celcius': 42.0, 'Kelvin': 315.4}},
          {'Confidence': 73.0,
           'Datetime': '2017-09-21T05:11:47',
           'Location': {'Latitude': -37.227, 'Longitude': 142.138},
           'Power': 51.0,
           'SurfaceTemperature': {'Celcius': 39.0, 'Kelvin': 312.7}}],
 'MAX': '71.8*',
 'MIN': '51.6',
 'MaxWindSpeed': 22.9,
 'Precipitation': '0.00I',
 'RelativeHumidity': 47.3,
 'Station': 948702,
 'WindSpeedKnots': 8.6}
{'AirTemperature': {'Celcius': 20, 'Kelvin': 68.0},
 'Date': '2017-09-22',
 'Fire': [{'Confidence': 71.0,
           'Datetime': '2017-09-22T04:17:10',
           'Location': {'Latitude': -36.012, 'Longitude': 141.341},
           'Power': 11.6,
           'SurfaceTemperature': {'Celcius': 45.0, 'Kelvin': 318.3}}],
 'MAX': '74.

           'Location': {'Latitude': -37.461, 'Longitude': 148.105},
           'Power': 34.1,
           'SurfaceTemperature': {'Celcius': 42.0, 'Kelvin': 316.1}},
          {'Confidence': 50.0,
           'Datetime': '2017-10-02T23:44:31',
           'Location': {'Latitude': -37.466, 'Longitude': 148.1},
           'Power': 10.9,
           'SurfaceTemperature': {'Celcius': 29.0, 'Kelvin': 302.2}},
          {'Confidence': 59.0,
           'Datetime': '2017-10-02T04:53:10',
           'Location': {'Latitude': -37.475, 'Longitude': 148.134},
           'Power': 62.0,
           'SurfaceTemperature': {'Celcius': 40.0, 'Kelvin': 313.9}},
          {'Confidence': 83.0,
           'Datetime': '2017-10-02T04:53:10',
           'Location': {'Latitude': -37.47, 'Longitude': 148.089},
           'Power': 117.5,
           'SurfaceTemperature': {'Celcius': 50.0, 'Kelvin': 323.7}},
          {'Confidence': 68.0,
           'Datetime': '2017-10-02T00:39:34',
           'Location': {'Latitude': -3

           'Power': 31.1,
           'SurfaceTemperature': {'Celcius': 58.0, 'Kelvin': 332.0}}],
 'MAX': '62.1*',
 'MIN': '42.3',
 'MaxWindSpeed': 14.0,
 'Precipitation': '0.00G',
 'RelativeHumidity': 43.3,
 'Station': 948702,
 'WindSpeedKnots': 9.4}
{'AirTemperature': {'Celcius': 13, 'Kelvin': 55.4},
 'Date': '2017-11-06',
 'MAX': '60.6*',
 'MIN': '49.5',
 'MaxWindSpeed': 19.0,
 'Precipitation': '0.02G',
 'RelativeHumidity': 44.1,
 'Station': 948702,
 'WindSpeedKnots': 12.9}
{'AirTemperature': {'Celcius': 12, 'Kelvin': 53.6},
 'Date': '2017-11-07',
 'MAX': '57.9*',
 'MIN': '51.3',
 'MaxWindSpeed': 15.9,
 'Precipitation': '0.01G',
 'RelativeHumidity': 42.7,
 'Station': 948702,
 'WindSpeedKnots': 10.0}
{'AirTemperature': {'Celcius': 12, 'Kelvin': 53.6},
 'Date': '2017-11-08',
 'Fire': [{'Confidence': 60.0,
           'Datetime': '2017-11-08T05:12:08',
           'Location': {'Latitude': -36.538000000000004, 'Longitude': 141.129},
           'Power': 21.0,
           'SurfaceTemperature'

           'Power': 12.0,
           'SurfaceTemperature': {'Celcius': 44.0, 'Kelvin': 317.5}},
          {'Confidence': 100.0,
           'Datetime': '2017-11-29T03:52:10',
           'Location': {'Latitude': -37.606, 'Longitude': 149.312},
           'Power': 100.7,
           'SurfaceTemperature': {'Celcius': 87.0, 'Kelvin': 360.7}},
          {'Confidence': 52.0,
           'Datetime': '2017-11-29T03:53:16',
           'Location': {'Latitude': -34.282, 'Longitude': 142.121},
           'Power': 39.3,
           'SurfaceTemperature': {'Celcius': 54.0, 'Kelvin': 327.8}},
          {'Confidence': 68.0,
           'Datetime': '2017-11-29T13:17:24',
           'Location': {'Latitude': -37.605, 'Longitude': 149.308},
           'Power': 22.7,
           'SurfaceTemperature': {'Celcius': 43.0, 'Kelvin': 316.9}},
          {'Confidence': 100.0,
           'Datetime': '2017-11-29T13:17:24',
           'Location': {'Latitude': -37.608000000000004, 'Longitude': 149.292},
           'Power': 4

{'AirTemperature': {'Celcius': 17, 'Kelvin': 62.6},
 'Date': '2017-12-10',
 'Fire': [{'Confidence': 67.0,
           'Datetime': '2017-12-10T00:56:57',
           'Location': {'Latitude': -34.289, 'Longitude': 141.71200000000002},
           'Power': 12.3,
           'SurfaceTemperature': {'Celcius': 54.0, 'Kelvin': 327.7}},
          {'Confidence': 50.0,
           'Datetime': '2017-12-10T00:57:59',
           'Location': {'Latitude': -37.986999999999995, 'Longitude': 144.005},
           'Power': 10.4,
           'SurfaceTemperature': {'Celcius': 38.0, 'Kelvin': 311.2}}],
 'MAX': '69.1*',
 'MIN': '57.6',
 'MaxWindSpeed': 14.0,
 'Precipitation': '0.00I',
 'RelativeHumidity': 53.5,
 'Station': 948702,
 'WindSpeedKnots': 7.3}
{'AirTemperature': {'Celcius': 17, 'Kelvin': 62.6},
 'Date': '2017-12-11',
 'MAX': '67.5*',
 'MIN': '59.4*',
 'MaxWindSpeed': 15.0,
 'Precipitation': '0.00I',
 'RelativeHumidity': 52.0,
 'Station': 948702,
 'WindSpeedKnots': 8.8}
{'AirTemperature': {'Celcius': 19, 

### Putting data in database

In [157]:
#deleting any previous records
coll.delete_many({})

In [158]:
#inserting the new records
result = coll.insert_many(data)

### Check inserted data

In [159]:
# checking data
# c = coll.find()
# for x in c:
#     pp.pprint(x)

In [160]:
#comparing counts of objects with dates
coll.find().count()

366

In [161]:
# comparing counts of dates with objects
len(df.Date.unique())

366

### Queries

Write queries to answer the Tasks A2-A8 on the new database. You need to write the queries as a python program using pymongo library in Jupyter Notebook. Please choose
ONE of the Task A2-A8 and implement it in parallel using Pool library.

###### A2

In [162]:
for i in coll.find( { 'Date': "2017-12-15" }, { 'Fire': 0, '_id': 0} ):
    pp.pprint(i)

{'AirTemperature': {'Celcius': 18, 'Kelvin': 64.4},
 'Date': '2017-12-15',
 'MAX': '74.5*',
 'MIN': '53.1',
 'MaxWindSpeed': 14.0,
 'Precipitation': '0.00I',
 'RelativeHumidity': 52.0,
 'Station': 948702,
 'WindSpeedKnots': 7.1}


###### A3

In [170]:
for i in coll.aggregate(
    [
        {'$unwind' : "$Fire" },
        {'$match': {'$and':[{'Fire.SurfaceTemperature.Celcius':{'$gt':65}},{'Fire.SurfaceTemperature.Celcius':{'$lt':100}}]}},
        {'$group' : { '_id' : "$Date", 'Fire': { '$push': {'Location':'$Fire.Location', 'Confidence':'$Fire.Confidence'} } } },
        {"$project": {"Fire": '$Fire', "_id": 0}},
        {'$unwind' : "$Fire" }   
        
    ]
):
    pp.pprint(i)

{'Fire': {'Confidence': 78.0,
          'Location': {'Latitude': -37.966, 'Longitude': 145.05100000000002}}}
{'Fire': {'Confidence': 86.0,
          'Location': {'Latitude': -35.543, 'Longitude': 143.316}}}
{'Fire': {'Confidence': 93.0,
          'Location': {'Latitude': -38.056999999999995, 'Longitude': 144.211}}}
{'Fire': {'Confidence': 90.0,
          'Location': {'Latitude': -37.624, 'Longitude': 149.314}}}
{'Fire': {'Confidence': 95.0,
          'Location': {'Latitude': -37.613, 'Longitude': 149.305}}}
{'Fire': {'Confidence': 93.0,
          'Location': {'Latitude': -37.875, 'Longitude': 142.51}}}
{'Fire': {'Confidence': 92.0,
          'Location': {'Latitude': -37.95, 'Longitude': 142.366}}}
{'Fire': {'Confidence': 100.0,
          'Location': {'Latitude': -36.282, 'Longitude': 146.157}}}
{'Fire': {'Confidence': 97.0,
          'Location': {'Latitude': -37.61, 'Longitude': 149.30700000000002}}}
{'Fire': {'Confidence': 100.0,
          'Location': {'Latitude': -37.606, 'Longitude'

{'Fire': {'Confidence': 93.0,
          'Location': {'Latitude': -36.8879, 'Longitude': 141.6647}}}
{'Fire': {'Confidence': 100.0,
          'Location': {'Latitude': -35.8768, 'Longitude': 145.4627}}}
{'Fire': {'Confidence': 93.0,
          'Location': {'Latitude': -36.4899, 'Longitude': 142.2805}}}
{'Fire': {'Confidence': 93.0,
          'Location': {'Latitude': -36.663000000000004, 'Longitude': 141.3038}}}
{'Fire': {'Confidence': 100.0,
          'Location': {'Latitude': -37.5701, 'Longitude': 142.6497}}}
{'Fire': {'Confidence': 92.0,
          'Location': {'Latitude': -37.9284, 'Longitude': 143.108}}}
{'Fire': {'Confidence': 98.0,
          'Location': {'Latitude': -38.0509, 'Longitude': 143.9423}}}
{'Fire': {'Confidence': 91.0,
          'Location': {'Latitude': -37.6964, 'Longitude': 142.8583}}}
{'Fire': {'Confidence': 94.0,
          'Location': {'Latitude': -37.7274, 'Longitude': 142.9536}}}
{'Fire': {'Confidence': 90.0,
          'Location': {'Latitude': -36.7101, 'Longitude': 

{'Fire': {'Confidence': 95.0,
          'Location': {'Latitude': -36.6537, 'Longitude': 143.8781}}}
{'Fire': {'Confidence': 99.0,
          'Location': {'Latitude': -36.5438, 'Longitude': 144.7548}}}
{'Fire': {'Confidence': 90.0,
          'Location': {'Latitude': -36.094, 'Longitude': 145.7646}}}
{'Fire': {'Confidence': 96.0,
          'Location': {'Latitude': -36.0594, 'Longitude': 141.5907}}}
{'Fire': {'Confidence': 100.0,
          'Location': {'Latitude': -36.0576, 'Longitude': 141.6122}}}
{'Fire': {'Confidence': 93.0,
          'Location': {'Latitude': -36.388000000000005, 'Longitude': 141.8309}}}
{'Fire': {'Confidence': 71.0,
          'Location': {'Latitude': -37.522, 'Longitude': 143.3874}}}
{'Fire': {'Confidence': 92.0,
          'Location': {'Latitude': -38.0715, 'Longitude': 142.9591}}}
{'Fire': {'Confidence': 100.0,
          'Location': {'Latitude': -38.0693, 'Longitude': 142.9722}}}
{'Fire': {'Confidence': 91.0,
          'Location': {'Latitude': -38.0674, 'Longitude': 1

###### A4

In [163]:
for i in coll.find({'Date':{'$in': ["2017-12-15", "2017-12-16"]}}, { '_id': 0,'Date': 1, 'AirTemperature': 1, 'RelativeHumidity': 1, 'MaxWindSpeed': 1, 'Fire.SurfaceTemperature': 1}):
    pp.pprint(i)

{'AirTemperature': {'Celcius': 18, 'Kelvin': 64.4},
 'Date': '2017-12-15',
 'Fire': [{'SurfaceTemperature': {'Celcius': 40.0, 'Kelvin': 313.4}},
          {'SurfaceTemperature': {'Celcius': 42.0, 'Kelvin': 315.9}},
          {'SurfaceTemperature': {'Celcius': 36.0, 'Kelvin': 309.8}},
          {'SurfaceTemperature': {'Celcius': 38.0, 'Kelvin': 311.9}}],
 'MaxWindSpeed': 14.0,
 'RelativeHumidity': 52.0}
{'AirTemperature': {'Celcius': 18, 'Kelvin': 64.4},
 'Date': '2017-12-16',
 'Fire': [{'SurfaceTemperature': {'Celcius': 57.0, 'Kelvin': 330.7}},
          {'SurfaceTemperature': {'Celcius': 64.0, 'Kelvin': 337.8}},
          {'SurfaceTemperature': {'Celcius': 55.0, 'Kelvin': 329.0}},
          {'SurfaceTemperature': {'Celcius': 48.0, 'Kelvin': 322.0}},
          {'SurfaceTemperature': {'Celcius': 73.0, 'Kelvin': 346.3}},
          {'SurfaceTemperature': {'Celcius': 60.0, 'Kelvin': 333.8}},
          {'SurfaceTemperature': {'Celcius': 56.0, 'Kelvin': 329.7}},
          {'SurfaceTemperatur

###### A5

In [177]:
for i in coll.find({"Fire.Confidence": {"$gt": 80, "$lt": 100}}, { '_id': 0,'Fire.Datetime': 1, 'AirTemperature': 1, 'Fire.SurfaceTemperature': 1, 'Fire.Confidence': 1}):
    pp.pprint(i)

{'AirTemperature': {'Celcius': 20, 'Kelvin': 68.0},
 'Fire': [{'Confidence': 87.0,
           'Datetime': '2017-03-06T05:06:30',
           'SurfaceTemperature': {'Celcius': 62.0, 'Kelvin': 335.4}},
          {'Confidence': 85.0,
           'Datetime': '2017-03-06T05:06:20',
           'SurfaceTemperature': {'Celcius': 59.0, 'Kelvin': 332.2}}]}
{'AirTemperature': {'Celcius': 19, 'Kelvin': 66.2},
 'Fire': [{'Confidence': 88.0,
           'Datetime': '2017-03-07T04:16:10',
           'SurfaceTemperature': {'Celcius': 64.0, 'Kelvin': 337.8}}]}
{'AirTemperature': {'Celcius': 23, 'Kelvin': 73.4},
 'Fire': [{'Confidence': 54.0,
           'Datetime': '2017-03-09T03:57:00',
           'SurfaceTemperature': {'Celcius': 44.0, 'Kelvin': 317.7}},
          {'Confidence': 86.0,
           'Datetime': '2017-03-09T13:23:40',
           'SurfaceTemperature': {'Celcius': 41.0, 'Kelvin': 314.6}},
          {'Confidence': 78.0,
           'Datetime': '2017-03-09T03:56:50',
           'SurfaceTemperature

           'SurfaceTemperature': {'Celcius': 66.0, 'Kelvin': 339.4}},
          {'Confidence': 83.0,
           'Datetime': '2017-04-03T03:51:30',
           'SurfaceTemperature': {'Celcius': 63.0, 'Kelvin': 336.5}},
          {'Confidence': 66.0,
           'Datetime': '2017-04-03T03:52:10',
           'SurfaceTemperature': {'Celcius': 43.0, 'Kelvin': 316.5}},
          {'Confidence': 75.0,
           'Datetime': '2017-04-03T03:51:10',
           'SurfaceTemperature': {'Celcius': 49.0, 'Kelvin': 322.5}},
          {'Confidence': 85.0,
           'Datetime': '2017-04-03T03:53:30',
           'SurfaceTemperature': {'Celcius': 59.0, 'Kelvin': 332.5}},
          {'Confidence': 75.0,
           'Datetime': '2017-04-03T03:58:20',
           'SurfaceTemperature': {'Celcius': 49.0, 'Kelvin': 322.3}},
          {'Confidence': 71.0,
           'Datetime': '2017-04-03T03:53:50',
           'SurfaceTemperature': {'Celcius': 45.0, 'Kelvin': 319.1}},
          {'Confidence': 72.0,
           'Datet

           'SurfaceTemperature': {'Celcius': 61.0, 'Kelvin': 334.6}},
          {'Confidence': 84.0,
           'Datetime': '2017-04-04T04:32:50',
           'SurfaceTemperature': {'Celcius': 58.0, 'Kelvin': 331.9}},
          {'Confidence': 82.0,
           'Datetime': '2017-04-04T04:32:50',
           'SurfaceTemperature': {'Celcius': 55.0, 'Kelvin': 328.8}},
          {'Confidence': 54.0,
           'Datetime': '2017-04-04T04:32:50',
           'SurfaceTemperature': {'Celcius': 49.0, 'Kelvin': 323.0}},
          {'Confidence': 70.0,
           'Datetime': '2017-04-04T04:32:50',
           'SurfaceTemperature': {'Celcius': 48.0, 'Kelvin': 321.4}},
          {'Confidence': 97.0,
           'Datetime': '2017-04-04T04:33:00',
           'SurfaceTemperature': {'Celcius': 79.0, 'Kelvin': 353.0}},
          {'Confidence': 100.0,
           'Datetime': '2017-04-04T04:32:50',
           'SurfaceTemperature': {'Celcius': 123.0, 'Kelvin': 396.3}}]}
{'AirTemperature': {'Celcius': 17, 'Kelvin': 

          {'Confidence': 91.0,
           'Datetime': '2017-04-06T04:20:40',
           'SurfaceTemperature': {'Celcius': 68.0, 'Kelvin': 341.6}},
          {'Confidence': 97.0,
           'Datetime': '2017-04-06T04:20:40',
           'SurfaceTemperature': {'Celcius': 81.0, 'Kelvin': 354.2}},
          {'Confidence': 82.0,
           'Datetime': '2017-04-06T04:20:40',
           'SurfaceTemperature': {'Celcius': 56.0, 'Kelvin': 329.4}},
          {'Confidence': 94.0,
           'Datetime': '2017-04-06T04:20:40',
           'SurfaceTemperature': {'Celcius': 104.0, 'Kelvin': 377.5}},
          {'Confidence': 83.0,
           'Datetime': '2017-04-06T04:20:50',
           'SurfaceTemperature': {'Celcius': 63.0, 'Kelvin': 337.0}},
          {'Confidence': 58.0,
           'Datetime': '2017-04-06T04:20:40',
           'SurfaceTemperature': {'Celcius': 52.0, 'Kelvin': 325.7}},
          {'Confidence': 92.0,
           'Datetime': '2017-04-06T04:20:40',
           'SurfaceTemperature': {'Celci

          {'Confidence': 65.0,
           'Datetime': '2017-04-12T03:44:50',
           'SurfaceTemperature': {'Celcius': 42.0, 'Kelvin': 316.1}},
          {'Confidence': 65.0,
           'Datetime': '2017-04-12T03:44:40',
           'SurfaceTemperature': {'Celcius': 43.0, 'Kelvin': 316.2}},
          {'Confidence': 100.0,
           'Datetime': '2017-04-12T03:44:50',
           'SurfaceTemperature': {'Celcius': 98.0, 'Kelvin': 371.4}},
          {'Confidence': 68.0,
           'Datetime': '2017-04-12T03:44:40',
           'SurfaceTemperature': {'Celcius': 44.0, 'Kelvin': 317.6}},
          {'Confidence': 57.0,
           'Datetime': '2017-04-12T03:44:40',
           'SurfaceTemperature': {'Celcius': 40.0, 'Kelvin': 313.5}},
          {'Confidence': 87.0,
           'Datetime': '2017-04-12T03:44:40',
           'SurfaceTemperature': {'Celcius': 63.0, 'Kelvin': 336.3}},
          {'Confidence': 77.0,
           'Datetime': '2017-04-12T03:44:40',
           'SurfaceTemperature': {'Celci

...

          {'Confidence': 63.0,
           'Datetime': '2017-05-01T04:15:40',
           'SurfaceTemperature': {'Celcius': 41.0, 'Kelvin': 315.1}},
          {'Confidence': 62.0,
           'Datetime': '2017-05-01T04:14:40',
           'SurfaceTemperature': {'Celcius': 41.0, 'Kelvin': 314.6}},
          {'Confidence': 82.0,
           'Datetime': '2017-05-01T04:14:30',
           'SurfaceTemperature': {'Celcius': 55.0, 'Kelvin': 328.9}},
          {'Confidence': 77.0,
           'Datetime': '2017-05-01T04:20:40',
           'SurfaceTemperature': {'Celcius': 50.0, 'Kelvin': 323.6}},
          {'Confidence': 82.0,
           'Datetime': '2017-05-01T04:14:30',
           'SurfaceTemperature': {'Celcius': 55.0, 'Kelvin': 328.5}},
          {'Confidence': 83.0,
           'Datetime': '2017-05-01T04:14:20',
           'SurfaceTemperature': {'Celcius': 77.0, 'Kelvin': 350.3}},
          {'Confidence': 70.0,
           'Datetime': '2017-05-01T04:14:20',
           'SurfaceTemperature': {'Celciu

          {'Confidence': 91.0,
           'Datetime': '2017-05-04T04:44:40',
           'SurfaceTemperature': {'Celcius': 69.0, 'Kelvin': 342.2}},
          {'Confidence': 77.0,
           'Datetime': '2017-05-04T04:44:40',
           'SurfaceTemperature': {'Celcius': 51.0, 'Kelvin': 324.3}},
          {'Confidence': 87.0,
           'Datetime': '2017-05-04T04:44:40',
           'SurfaceTemperature': {'Celcius': 62.0, 'Kelvin': 336.1}},
          {'Confidence': 78.0,
           'Datetime': '2017-05-04T04:44:40',
           'SurfaceTemperature': {'Celcius': 51.0, 'Kelvin': 324.8}},
          {'Confidence': 92.0,
           'Datetime': '2017-05-04T04:44:40',
           'SurfaceTemperature': {'Celcius': 70.0, 'Kelvin': 344.1}},
          {'Confidence': 100.0,
           'Datetime': '2017-05-04T04:44:40',
           'SurfaceTemperature': {'Celcius': 112.0, 'Kelvin': 385.4}},
          {'Confidence': 59.0,
           'Datetime': '2017-05-04T04:44:40',
           'SurfaceTemperature': {'Celc

           'Datetime': '2017-05-10T04:08:10',
           'SurfaceTemperature': {'Celcius': 40.0, 'Kelvin': 313.5}},
          {'Confidence': 81.0,
           'Datetime': '2017-05-10T04:08:10',
           'SurfaceTemperature': {'Celcius': 55.0, 'Kelvin': 328.4}},
          {'Confidence': 87.0,
           'Datetime': '2017-05-10T04:08:10',
           'SurfaceTemperature': {'Celcius': 62.0, 'Kelvin': 335.3}}]}
{'AirTemperature': {'Celcius': 9, 'Kelvin': 48.2},
 'Fire': [{'Confidence': 84.0,
           'Datetime': '2017-05-11T04:50:40',
           'SurfaceTemperature': {'Celcius': 78.0, 'Kelvin': 352.0}},
          {'Confidence': 65.0,
           'Datetime': '2017-05-11T04:50:40',
           'SurfaceTemperature': {'Celcius': 42.0, 'Kelvin': 316.0}},
          {'Confidence': 94.0,
           'Datetime': '2017-05-11T04:50:40',
           'SurfaceTemperature': {'Celcius': 75.0, 'Kelvin': 348.2}},
          {'Confidence': 88.0,
           'Datetime': '2017-05-11T04:54:00',
           'SurfaceT

           'SurfaceTemperature': {'Celcius': 40.0, 'Kelvin': 313.4}},
          {'Confidence': 89.0,
           'Datetime': '2017-05-22T04:32:20',
           'SurfaceTemperature': {'Celcius': 65.0, 'Kelvin': 338.6}},
          {'Confidence': 98.0,
           'Datetime': '2017-05-22T04:32:20',
           'SurfaceTemperature': {'Celcius': 82.0, 'Kelvin': 355.7}},
          {'Confidence': 64.0,
           'Datetime': '2017-05-22T04:32:20',
           'SurfaceTemperature': {'Celcius': 42.0, 'Kelvin': 315.9}},
          {'Confidence': 65.0,
           'Datetime': '2017-05-22T04:32:20',
           'SurfaceTemperature': {'Celcius': 42.0, 'Kelvin': 315.8}},
          {'Confidence': 96.0,
           'Datetime': '2017-05-22T04:32:20',
           'SurfaceTemperature': {'Celcius': 78.0, 'Kelvin': 352.1}},
          {'Confidence': 76.0,
           'Datetime': '2017-05-22T04:32:20',
           'SurfaceTemperature': {'Celcius': 55.0, 'Kelvin': 328.9}},
          {'Confidence': 91.0,
           'Datet

          {'Confidence': 100.0,
           'Datetime': '2017-10-03T03:58:14',
           'SurfaceTemperature': {'Celcius': 90.0, 'Kelvin': 363.8}},
          {'Confidence': 71.0,
           'Datetime': '2017-10-03T03:58:15',
           'SurfaceTemperature': {'Celcius': 55.0, 'Kelvin': 328.5}},
          {'Confidence': 90.0,
           'Datetime': '2017-10-03T03:58:15',
           'SurfaceTemperature': {'Celcius': 63.0, 'Kelvin': 336.5}},
          {'Confidence': 84.0,
           'Datetime': '2017-10-03T03:58:15',
           'SurfaceTemperature': {'Celcius': 55.0, 'Kelvin': 328.4}},
          {'Confidence': 77.0,
           'Datetime': '2017-10-03T03:58:28',
           'SurfaceTemperature': {'Celcius': 43.0, 'Kelvin': 316.9}},
          {'Confidence': 71.0,
           'Datetime': '2017-10-03T03:58:14',
           'SurfaceTemperature': {'Celcius': 50.0, 'Kelvin': 323.4}},
          {'Confidence': 82.0,
           'Datetime': '2017-10-03T03:58:15',
           'SurfaceTemperature': {'Celci

###### A6

In [231]:
for i in coll.aggregate(
    [
        {'$unwind' : "$Fire" },
        { '$sort' : { 'Fire.SurfaceTemperature' : -1 } },
        { "$limit": 10 }
        
    ]
):
    pp.pprint(i)

{'AirTemperature': {'Celcius': 15, 'Kelvin': 59.0},
 'Date': '2017-04-18',
 'Fire': {'Confidence': 100.0,
          'Datetime': '2017-04-18T04:52:00',
          'Location': {'Latitude': -38.1665, 'Longitude': 143.062},
          'Power': 239.8,
          'SurfaceTemperature': {'Celcius': 124.0, 'Kelvin': 397.5}},
 'MAX': '69.8*',
 'MIN': '46.8',
 'MaxWindSpeed': 9.9,
 'Precipitation': '0.00I',
 'RelativeHumidity': 56.1,
 'Station': 948701,
 'WindSpeedKnots': 5.1,
 '_id': ObjectId('5afe81590da56904bae8c6e7')}
{'AirTemperature': {'Celcius': 16, 'Kelvin': 60.8},
 'Date': '2017-04-04',
 'Fire': {'Confidence': 100.0,
          'Datetime': '2017-04-04T04:32:50',
          'Location': {'Latitude': -36.343, 'Longitude': 142.1986},
          'Power': 233.4,
          'SurfaceTemperature': {'Celcius': 123.0, 'Kelvin': 396.3}},
 'MAX': '73.4*',
 'MIN': '46.8',
 'MaxWindSpeed': 12.0,
 'Precipitation': '0.00I',
 'RelativeHumidity': 47.5,
 'Station': 948701,
 'WindSpeedKnots': 5.4,
 '_id': ObjectId(

###### A7

In [252]:
for i in coll.aggregate([
    {'$unwind': '$Fire'},
    {'$group': {'_id': '$Date', 'count': {'$sum': 1}}}]):
    print(i)

{'_id': '2017-12-27', 'count': 4}
{'_id': '2017-12-24', 'count': 1}
{'_id': '2017-12-15', 'count': 4}
{'_id': '2017-12-14', 'count': 1}
{'_id': '2017-12-12', 'count': 1}
{'_id': '2017-12-10', 'count': 2}
{'_id': '2017-12-09', 'count': 4}
{'_id': '2017-11-29', 'count': 8}
{'_id': '2017-11-28', 'count': 1}
{'_id': '2017-11-23', 'count': 5}
{'_id': '2017-11-22', 'count': 2}
{'_id': '2017-11-14', 'count': 3}
{'_id': '2017-11-13', 'count': 5}
{'_id': '2017-11-11', 'count': 4}
{'_id': '2017-11-09', 'count': 10}
{'_id': '2017-11-05', 'count': 4}
{'_id': '2017-10-27', 'count': 5}
{'_id': '2017-10-18', 'count': 6}
{'_id': '2017-10-16', 'count': 1}
{'_id': '2017-10-15', 'count': 3}
{'_id': '2017-10-10', 'count': 3}
{'_id': '2017-10-09', 'count': 1}
{'_id': '2017-10-17', 'count': 5}
{'_id': '2017-10-07', 'count': 1}
{'_id': '2017-10-03', 'count': 18}
{'_id': '2017-12-08', 'count': 5}
{'_id': '2017-10-23', 'count': 1}
{'_id': '2017-10-02', 'count': 7}
{'_id': '2017-09-29', 'count': 2}
{'_id': '201

###### A8

In [266]:
for i in coll.aggregate([
    {'$unwind': '$Fire'},
    {'$group': {'_id': '$Date', 'AST': {'$avg': '$Fire.SurfaceTemperature.Celcius'}}}]):
    print(i)

{'_id': '2017-12-27', 'AST': 62.75}
{'_id': '2017-12-24', 'AST': 32.0}
{'_id': '2017-12-15', 'AST': 39.0}
{'_id': '2017-12-14', 'AST': 70.0}
{'_id': '2017-12-12', 'AST': 44.0}
{'_id': '2017-12-10', 'AST': 46.0}
{'_id': '2017-12-09', 'AST': 58.25}
{'_id': '2017-11-29', 'AST': 60.625}
{'_id': '2017-11-28', 'AST': 42.0}
{'_id': '2017-11-23', 'AST': 58.8}
{'_id': '2017-11-22', 'AST': 61.5}
{'_id': '2017-11-14', 'AST': 52.0}
{'_id': '2017-11-13', 'AST': 47.0}
{'_id': '2017-11-11', 'AST': 46.25}
{'_id': '2017-11-09', 'AST': 61.3}
{'_id': '2017-11-05', 'AST': 58.5}
{'_id': '2017-10-27', 'AST': 50.4}
{'_id': '2017-10-18', 'AST': 52.166666666666664}
{'_id': '2017-10-16', 'AST': 36.0}
{'_id': '2017-10-15', 'AST': 72.66666666666667}
{'_id': '2017-10-10', 'AST': 53.333333333333336}
{'_id': '2017-10-09', 'AST': 44.0}
{'_id': '2017-10-17', 'AST': 51.6}
{'_id': '2017-10-07', 'AST': 42.0}
{'_id': '2017-10-03', 'AST': 50.0}
{'_id': '2017-12-08', 'AST': 50.6}
{'_id': '2017-10-23', 'AST': 38.0}
{'_id': '

In [270]:
data

[{'AirTemperature': {'Celcius': 19, 'Kelvin': 66.2},
  'Date': '2016-12-31',
  'MAX': '72.0*',
  'MIN': '61.9*',
  'MaxWindSpeed': 11.1,
  'Precipitation': '0.00I',
  'RelativeHumidity': 56.8,
  'Station': 948700,
  'WindSpeedKnots': 7.9,
  '_id': ObjectId('5afe81590da56904bae8c67c')},
 {'AirTemperature': {'Celcius': 15, 'Kelvin': 59.0},
  'Date': '2017-01-02',
  'MAX': '72.0*',
  'MIN': '58.3',
  'MaxWindSpeed': 13.0,
  'Precipitation': '0.02G',
  'RelativeHumidity': 50.7,
  'Station': 948700,
  'WindSpeedKnots': 9.2,
  '_id': ObjectId('5afe81590da56904bae8c67d')},
 {'AirTemperature': {'Celcius': 16, 'Kelvin': 60.8},
  'Date': '2017-01-03',
  'MAX': '68.4*',
  'MIN': '54.3*',
  'MaxWindSpeed': 15.0,
  'Precipitation': '0.00G',
  'RelativeHumidity': 53.6,
  'Station': 948700,
  'WindSpeedKnots': 8.1,
  '_id': ObjectId('5afe81590da56904bae8c67e')},
 {'AirTemperature': {'Celcius': 24, 'Kelvin': 75.2},
  'Date': '2017-01-04',
  'MAX': '87.3*',
  'MIN': '54',
  'MaxWindSpeed': 14.0,
  'Pre

In [313]:
import collections

result = collections.defaultdict(list)

for d in data:
    result[d['Date']].append(d)

result_list = result.values()

In [314]:
# list(result)[0]

'2016-12-31'

In [315]:
# list(result.keys())[0]

'2016-12-31'

In [320]:
# dic = {}
# for key, value in result.items(): # For each data record (list), perform the following:
#     # Get the hash key of the inputs Date
#     h = s_hash(key, 4) 

#     #add records by hash key into dictionary
#     dic.setdefault(h, [])
#     dic[h].append(value)

In [319]:
# dic.()

## Parallel

Write an algorithm to search climate data for the records on 15th December 2017 . Justify
your choice of the data partition technique and search technique you have used.





Solution:

-Partioning
Hash Partitioning: since the search will be on the partioning attribute Range/Hash is better than other. Also since the search is an exact search, Hash partioning is even better than Range partioning as it utilizes only the processor that holds the data for that value. 

-Search:
binary search as data can be sorted with dates as integers, sorting data on integers is easy and hence binary search would perform faster than linear search. 

#### Data partition

##### - Defining hash function

In [321]:
# Define a simple hash function.
def s_hash(x, n):
    """
    Define a simple hash function for demonstration
    Arguments:
    x -- an input date as string
    n -- the number of processors
    Return:
    result -- the hash value of x
    """
    ### START CODE HERE ###
    date_parse = x.split('-')
    result = int(date_parse[1])%n #hash key by months
    
    ### END CODE HERE ###
    return result

##### - Defining hash partition

In [322]:
# Hash data partitionining function.
# We will use the "s_hash" function defined above to realise this partitioning
def h_partition(data, n):
    
    """
    Perform hash data partitioning on data
    Arguments:
    data -- an input dataset which is a list of lists
    n -- the number of processors
    Return:
    result -- the paritioned subsets of data
    """
    
    dic = {} # create a dictionary
    
    for key, value in result.items(): # For each data record (list), perform the following:
        
        # Get the hash key of the inputs Date
        h = s_hash(key, n) 
        
        #add records by hash key into dictionary
        dic.setdefault(h, [])
        dic[h].append(value)
    
    return dic

In [385]:
# h_partition(data,4)

In [422]:
# dic[key$#][block#][0]['Date'] <- To fetch date
# dic[0][0][0]['Date']#[2][0]['Date']

In [421]:
# for i in range(len(h_partition(data,4))):
#     for j in range(len(h_partition(data,4)[i])):
#         print(h_partition(data,4)[i][j][0]['Date'])
#     print('------------------------------------------')

#### Binary Search Algo

##### - function to change date into integer for comparisor and sorting

In [374]:
def date_to_int(date):
    
    """
    converts a date string of the form 'yyyy-mm-dd' to int of the form 'yyyymmdd'
    arguments:
    date -- the date that needs to be converted to int form 
    return:
    result -- int representation of the date
    """
    
    result = int(date.replace('-',''))
    
    return result

##### -binary search algo

In [412]:
# Binary search function
def binary_search(data, key):
    
    """
    Perform binary search on data for the given key by sorting it by date first
    Arguments:
    data -- an input dataset which is a list
    key -- an query record
    Return:
    result -- the position of searched record
    """

    matched_record = None
    position = -1 # not found position
    lower = 0
    middle = 0
    upper = len(data)-1
    key = date_to_int(key)
    
# Data is already sorted by Dates
    
    ### START CODE HERE ###
    
    while (lower <= upper):
        
        # calculate middle: the half of lower and upper
        middle = int((lower + upper)/2)
        
        #compare date of rec to key
        if date_to_int(data[middle][0]['Date']) == key:
            position = middle
            matched_record = data[middle]
            break
        elif key > date_to_int(data[middle][0]['Date']):
            lower = middle+1
        else:
            upper = middle-1
            
        
    ### END CODE HERE ###
    return matched_record # ,position

In [420]:
# binary_search(dic[0],'2017-12-15')

#### multiprocessing

In [426]:
# Parallel searching algorithm for exact match
def parallel_search_exact(data, query, n_processor):

    """
    Perform parallel search for exact match on data for the given key
    Arguments:
    data -- an input dataset which is a list of lists
    query -- a query record ie. date to search on
    n_processor -- the number of parallel processors
    Return:
    results -- the matched record information
    """

    #results = []
    
    # Pool: a Python method enabling parallel processing.
    # We need to set the number of processes to n_processor,
    # which means that the Pool class will only allow 'n_processor' processes
    # running at the same time.
    
    pool = Pool(processes=n_processor)
    
    # Perform data partitioning first
    DD = h_partition(data, n_processor)
    # Each element in DD has a pair (hash key: records)
    query_hash = s_hash(query, n_processor)
    d = list(DD[query_hash])
    result = pool.apply(binary_search, [d, query])
    #results.append(result)
    pool.close() #close pool
    
    
    """
    The method above 'pool.apply()' will lock the function program until all a process
    is finished. Alternatively, we can use the 'pool.apply_sync()' method
    to spawn one process for each CPU core on your machine.
    """

    ### END CODE HERE ###
    
    return result#results

### Solution

In [427]:
parallel_search_exact(dic, '2017-12-15', 4)

[{'AirTemperature': {'Celcius': 18, 'Kelvin': 64.4},
  'Date': '2017-12-15',
  'Fire': [{'Confidence': 65.0,
    'Datetime': '2017-12-15T01:16:23',
    'Location': {'Latitude': -38.038000000000004, 'Longitude': 142.986},
    'Power': 32.8,
    'SurfaceTemperature': {'Celcius': 40.0, 'Kelvin': 313.4}},
   {'Confidence': 92.0,
    'Datetime': '2017-12-15T13:17:17',
    'Location': {'Latitude': -37.626999999999995, 'Longitude': 149.33},
    'Power': 23.1,
    'SurfaceTemperature': {'Celcius': 42.0, 'Kelvin': 315.9}},
   {'Confidence': 78.0,
    'Datetime': '2017-12-15T13:17:17',
    'Location': {'Latitude': -37.658, 'Longitude': 149.339},
    'Power': 17.3,
    'SurfaceTemperature': {'Celcius': 36.0, 'Kelvin': 309.8}},
   {'Confidence': 51.0,
    'Datetime': '2017-12-15T13:17:17',
    'Location': {'Latitude': -37.623000000000005,
     'Longitude': 149.32299999999998},
    'Power': 17.8,
    'SurfaceTemperature': {'Celcius': 38.0, 'Kelvin': 311.9}}],
  'MAX': '74.5*',
  'MIN': '53.1',
  'M